# ETL

In [ ]:
import pandas as pd
import numpy as np
import ast
import json
import nltk
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

Función `load_json` recibe el nombre del archivo en formato JSON y realiza la carga

In [ ]:
def load_json(file):
    with open(file,'r',encoding='utf-8') as file:
        data_list = []
        for line in file:
            try:
                data_list.append(json.loads(line))
            except ValueError:
                data =  ast.literal_eval(line)
                if isinstance(data, dict):
                    data_list.append(data)
    return pd.DataFrame(data_list)

Carga de los archivos:
- `australian_user_reviews.json`
- `australian_users_items.json`
- `output_steam_games.json`

Los cuales son cargados en los DataFrames `reviews`, `items` y `games`

In [ ]:
reviews = load_json('../dataset/australian_user_reviews.json')
items = load_json('../dataset/australian_users_items.json')
games = load_json('../dataset/output_steam_games.json')

Limpieza de datos para los DataFrames:
- reviews: Puede notarse que no contiene nulos y la columna `reviews` contiene las recomendaciones de de los juegos que el usuario ha rankeado, por lo que dicha columna esta anidada

In [ ]:
reviews.sample()

In [ ]:
reviews.info()

- items: Este DataFrame no contiene registros nulos y es notorio que la columna `items` contiene los juegos de los usuarios, por lo que dicha columna esta anidada

In [ ]:
items.sample()

In [ ]:
items.info()

- games
En este DataFrame encontramos una gran cantidad de registros nulos, por lo que se proceden a eliminar los registros nulos de las columnas `app_name` y `title`. Ademas de lo anterior es notorio que la columna `price` contiene datos numéricos y cadenas de texto, por lo que se homogeniza el tipo de dato a tipo string, debido a las diferentes variantes encontradas en este campo y la columna `release_date` contiene fechas por lo que se le asigna ese tipo de dato.

El contenido de los campos
-  `genres`: Es una lista de los generos que abarca el videojuego.
- `tags` : Lista de palabras claves que describen  al videojuego.
- `specs` : Lista de caracteristicas de jugabilidad del videojuego

In [ ]:
games.sample()

In [ ]:
games.info()

In [ ]:
games = games.dropna(subset=['app_name','title'])
games['price'] = games['price'].astype(str)
games['release_date'] = pd.to_datetime(games['release_date'], format="%Y-%m-%d", errors='coerce')

In [ ]:
games.info()

Creación de DataFrame para la función `PlayTimeGenre`

In [ ]:
# Función que permite desanidar los registros contenidos en la columna items
def unnesting_items(df): 
    i = 0
    data_list = []
    while i <= len(df['user_id']) - 1:
        user_id = df['user_id'].iloc[i]
        steam_id = df['steam_id'].iloc[i]
        user_url = df['user_url'].iloc[i]
        lista = df['items'].iloc[i]
        for j in lista:
            j['user_id'] = user_id
            j['steam_id'] = steam_id
            j['user_url'] = user_url
            data_list.append(j)
        i = i + 1
    return pd.DataFrame(data_list)
# Aplicamos la función a nuestro DataFrame items
items_unnesting = unnesting_items(items)
items_unnesting['item_id'] = items_unnesting['item_id'].astype(int) #  Convertimos el dato item_id de objeto a entero
items_unnesting['steam_id'] = items_unnesting['steam_id'].astype('Int64') # Convertimos  el dato 'steam_id' a entero

In [ ]:
games_exploded = games.explode('genres') # Desanidamos  los géneros en filas
# Reemplaza las cadenas que no sean fechas válidas por NaN
games_exploded['release_date'] = pd.to_datetime(games_exploded['release_date'], errors='coerce')
#Crear columna year
games_exploded['year'] = games_exploded['release_date'].dt.year
games_exploded['id'] = games_exploded['id'].astype('Int32') # Convertir columnas a int
# Combinar DF's games_explode con items coincidiendo por id    
merged_data = games_exploded.merge(items_unnesting, left_on='id', right_on='item_id', how='inner')
# Sumar playtime_forever por género y año
genre_playtime = merged_data.groupby(['genres','year'])['playtime_forever'].sum().reset_index()
# Ordenar de mayor a menor sobre 'playtime_forever'
ranking_genre = genre_playtime.sort_values(by='playtime_forever', ascending=False)
ranking_genre = ranking_genre.reset_index(drop=True) # Quitar el índice
ranking_genre['year'] = ranking_genre['year'].astype(int) #  Convertir a int los años 

In [ ]:
ranking_genre.to_parquet('../dataset/genres_playtime.parquet')

Creación del DataFrame para la función `UserForGenre`

In [ ]:
merged_data = items_unnesting.merge(games_exploded, left_on='item_id',right_on='id', how='inner')
user_playtime_genre = merged_data.groupby(['genres','user_id','year'])['playtime_forever'].sum().reset_index()
user_playtime_genre['year'] = user_playtime_genre['year'].astype(int)

In [ ]:
user_playtime_genre.to_parquet('../dataset/user_playtime2genres.parquet')

Creación del DataFrame `UsersRecommend`

Desanidar columna reviews del DataFrame reviews

In [ ]:
reviews = reviews.explode(column=['reviews']).reset_index()
reviews = reviews.drop('index',axis=1)
reviews_list = []
i = 0
while i <= len(reviews['user_id'])-1:
    dic = {'user_id':reviews['user_id'].loc[i]}
    if isinstance(reviews['reviews'].loc[i],dict):
        dic.update(reviews['reviews'].loc[i])
        reviews_list.append(dic)
    i  += 1
reviews = pd.DataFrame(reviews_list)

Homogenizar formato de fecha para la columna posted_date y last_edited

In [ ]:
def convertir_fecha(fecha):
    # Utilizar expresión regular para extraer componentes de la fecha
    match = re.match(r"Posted (\w+) (\d+), (\d+)", fecha)
    if match:
        mes_str, dia_str, anio_str = match.groups()
        # Mapear nombres de meses a números
        meses = {
            'January': '01', 'February': '02', 'March': '03', 'April': '04',
            'May': '05', 'June': '06', 'July': '07', 'August': '08',
            'September': '09', 'October': '10', 'November': '11', 'December': '12'
        }
        # Crear la fecha en el nuevo formato
        nueva_fecha = f"{anio_str}-{meses[mes_str]}-{dia_str.zfill(2)}"
        return nueva_fecha
    else:
        return None
reviews['posted_date'] = reviews['posted'].apply(convertir_fecha)
reviews['posted_date'] = pd.to_datetime(reviews['posted_date'])
reviews = reviews.drop(['posted'], axis=1)

In [ ]:
def convertir_fecha(fecha):
    # Utilizar expresión regular para extraer componentes de la fecha
    match = re.match(r"Last edited (\w+) (\d+), (\d+)", fecha)
    if match:
        mes_str, dia_str, anio_str = match.groups()
        # Mapear nombres de meses a números
        meses = {
            'January': '01', 'February': '02', 'March': '03', 'April': '04',
            'May': '05', 'June': '06', 'July': '07', 'August': '08',
            'September': '09', 'October': '10', 'November': '11', 'December': '12'
        }
        # Crear la fecha en el nuevo formato
        nueva_fecha = f"{anio_str}-{meses[mes_str]}-{dia_str.zfill(2)}"
        return nueva_fecha
    else:
        return None

reviews['last_edited'] = reviews['last_edited'].apply(convertir_fecha)
reviews['last_edited'] = pd.to_datetime(reviews['last_edited'])

La columna funny indica la cantidad de personas que les pareció graciosa las recomndación

In [ ]:
reviews['funny'] = reviews['funny'].str.replace('1 person found this review funny','1') # Reemplazar por 1
reviews['funny'] = reviews['funny'].str.replace(' people found this review funny','')   # Eliminar las palabras y dejar unicamente la cantidad
reviews['funny'] = reviews['funny'].str.strip()                                         # Eliminar espacios en blanco al principio y final
reviews['funny'] = pd.to_numeric(reviews['funny'], errors='coerce')                     # Convertir a dato tipo numérico
reviews['funny'] = reviews['funny'].fillna(0)                                           # Imputar  los valores NaN con 0 para que sea un número
reviews['funny'] = reviews['funny'].astype(int)                                         # Convertir a int

Función análisis de sentimiento con NLP

In [ ]:
sia = SentimentIntensityAnalyzer()
def sentiment_analisys(data):
    try:
        value = sia.polarity_scores(data)['compound']
        if  value >= 0.05:          # Sentimiento positivo
            return 2
        elif  value <= -0.05:       #  Sentimiento negativo
            return 0
        else:
            return 1                # Sentimiento  neutral
    except:
        return 1

reviews['sentiment_analysis'] = reviews['review'].apply(lambda x: sentiment_analisys(x))   # Aplicando NLP

In [ ]:
reviews.to_parquet('../dataset/reviews.parquet')

In [ ]:
reviews.sample()

In [ ]:
title_games = games[['title','id']]
user_recommend = reviews[['item_id','recommend','posted_date','sentiment_analysis']]
user_recommend = user_recommend.merge(title_games,left_on='item_id', right_on='id',how='inner')
user_recommend['year'] = user_recommend['posted_date'].dt.year
user_recommend.drop(columns=['id','posted_date'],inplace=True)
user_recommend = user_recommend.dropna(subset=['year'])
user_recommend['year']  = user_recommend['year'].astype(int)

In [ ]:
user_recommend.to_parquet('../dataset/user_recommend.parquet')

Creacion de DataFrame recommend_reviews

In [ ]:
# Función para concatenar columnas omitiendo nulos
def concat_cols(row):
    result = ''
    for col in range(len(row)-1):
        value = row[col]
        if pd.notnull(value):
            result += str(value)
    return result

In [ ]:
data = {}
for idx, row in reviews.iterrows():
    item_id = row['item_id']
    review = row['review']
    if item_id in data:
        data[item_id].append(review)
    else:
        data[item_id] = [review]

data_reviews = pd.DataFrame.from_dict(data, orient='index')
data_reviews = data_reviews.reset_index().rename(columns={'index': 'id'})
data_reviews['reviews'] = data_reviews.apply(concat_cols, axis=1)
data_reviews = data_reviews.drop(data_reviews.columns[range(1, len(data_reviews.columns)-1)], axis=1)
data_reviews['reviews'] = data_reviews.apply(concat_cols, axis=1)
data_reviews = data_reviews.drop(data_reviews.columns[range(1, len(data_reviews.columns)-1)], axis=1)

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)  # Se eligen 1000 palabras claves para vectorizar
# Aplicar el vectorizador a los textos de las reviews
vectores_reviews = vectorizer.fit_transform(data_reviews['reviews'])

df_vectores_reviews = pd.concat([data_reviews[['id', 'reviews']], pd.DataFrame(vectores_reviews.toarray(), columns=[f"feature_{i}" for i in range(vectores_reviews.shape[1])])], axis=1)
df_vectores_reviews = df_vectores_reviews.drop(['reviews'], axis=1)
df_vectores_reviews['id'] = df_vectores_reviews['id'].astype(int)
# Guardar el DataFrame en formato Parquet
df_vectores_reviews.to_parquet('../dataset/reviews_per_item.parquet', engine='pyarrow', compression='snappy')